# Format the soundings and save variables of interest in Tables folder

In [10]:
import numpy as np
import xarray as xr
from metpy.units import units
from metpy.calc import mixing_ratio
from tabulate import tabulate

# Path to your NetCDF files
path = 'C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/sounding/soundings/*.nc'
data = xr.open_mfdataset(path, combine='by_coords')


# Extract the variables needed for the sounding
p = (data['bar_pres'][:, :] * 10.0)
rh = (data['rh'][:, :]) 
T = data['temp'][:, :] 
Td = data['dp'][:, :]
wdir = data['wdir'][:, :]
wspd = (data['wspd'][:, :])
height = (data['bar_pres']['height'].values * 1000.0)
precip = data['precip'][:]

# Calculate additional variables needed for the sounding
#w = mixing_ratio(p, Td)

# Loop over all timeframes and save the tables with the appropriate names
for i in range(len(data['time'])):
    # Extract the variables for this timeframe
    p_i = p[i, :]
    rh_i = rh[i, :]
    T_i = T[i, :]
    Td_i = Td[i, :]
    wdir_i = wdir[i, :]
    wspd_i = wspd[i, :]
    
    # Combine the data into a table
    data_table_i = np.vstack((height, p_i, T_i, Td_i, rh_i, wdir_i, wspd_i)).T
    
    # Remove rows with NaN values
    mask_i = np.isnan(data_table_i).any(axis=1)
    data_table_i = data_table_i[~mask_i]
    
    # Save the modified data table as a txt file with the appropriate name
    #time_str = data['time'][i].strftime('%H%M%S')
    filename = 'C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/sounding/Tables/data-' + str(i) + '.txt'
    headers = ['HGHT', 'PRES', 'TEMP', 'DWPT', 'RH', 'DIR', 'SPD']
    np.savetxt(filename, data_table_i, header='\t'.join(headers), fmt='%0.2f', delimiter='\t')
    
    # Print the table for this timeframe
    #table_str = tabulate(data_table_i, headers=headers, tablefmt='plain')
    #print('Table for timeframe {}:'.format(time_str))
    #print(table_str)

In [11]:
data

<xarray.Dataset>
Dimensions:            (time: 132480, height: 332)
Coordinates:
  * time               (time) datetime64[ns] 2022-06-01T00:00:30 ... 2022-08-...
  * height             (height) float32 0.008 0.028 0.048 ... 39.51 40.01 40.51
Data variables: (12/39)
    base_time          (time) datetime64[ns] 2022-06-01 ... 2022-08-31
    time_offset        (time) datetime64[ns] dask.array<chunksize=(1440,), meta=np.ndarray>
    precip             (time) float32 dask.array<chunksize=(1440,), meta=np.ndarray>
    qc_precip          (time) int32 dask.array<chunksize=(1440,), meta=np.ndarray>
    temp               (time, height) float32 dask.array<chunksize=(1440, 332), meta=np.ndarray>
    qc_temp            (time, height) int32 dask.array<chunksize=(1440, 332), meta=np.ndarray>
    ...                 ...
    qc_rh_scaled       (time, height) int32 dask.array<chunksize=(1440, 332), meta=np.ndarray>
    aqc_rh_scaled      (time, height) int32 dask.array<chunksize=(1440, 332), meta=np.ndarray>
    vapor_source       (time, height) int32 dask.array<chunksize=(1440, 332), meta=np.ndarray>
    lat                (time) float32 29.67 29.67 29.67 ... 29.67 29.67 29.67
    lon                (time) float32 -95.06 -95.06 -95.06 ... -95.06 -95.06
    alt                (time) float32 8.0 8.0 8.0 8.0 8.0 ... 8.0 8.0 8.0 8.0
Attributes: (12/13)
    command_line:          idl -R -n interpolatedsonde -s hou -f M1 -b 202206...
    Conventions:           ARM-1.3
    process_version:       vap-interpolatedsonde-7.1-1.el7
    dod_version:           interpolatedsonde-c1-4.2
    input_datastreams:     hougriddedsondeM1.c0 : 3.2 : 20220530.000030-20220...
    site_id:               hou
    ...                    ...
    facility_id:           M1
    data_level:            c1
    location_description:  Tracking Aerosol Convection Interactions Experimen...
    datastream:            houinterpolatedsondeM1.c1
    doi:                   10.5439/1095316
    history:               created by user dsmgr on machine zinc at 2022-06-0...

# Calculate various sounding parameters for all the data in Tables folder and save the output to Output.txt

In [4]:
# Copyright (c) 2022 MetPy Developers.
# Distributed under the terms of the BSD 3-Clause License.
# SPDX-License-Identifier: BSD-3-Clause
"""
=============================
Sounding Calculation Examples
=============================

Use functions from `metpy.calc` to perform a number of calculations using sounding data.

The code below uses example data to perform many sounding calculations for a severe weather
event on May 22, 2011 from the Norman, OK sounding.
"""
import numpy as np
import pandas as pd

import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.units import units

###########################################
# Effective Shear Algorithm for use in Supercell Composite Calculation


def effective_layer(p, t, td, h, height_layer=False):
    """A function that determines the effective inflow layer for a convective sounding.

    Uses the default values of Thompason et al. (2004) for CAPE (100 J/kg) and CIN (-250 J/kg).

    Input:
      - p: sounding pressure with units
      - T: sounding temperature with units
      - Td: sounding dewpoint temperature with units
      - h: sounding heights with units

    Returns:
      - pbot/hbot, ptop/htop: pressure/height of the bottom level,
                              pressure/height of the top level
    """
    from metpy.calc import cape_cin, parcel_profile
    from metpy.units import units

    pbot = None

    for i in range(p.shape[0]):
        prof = parcel_profile(p[i:], t[i], td[i])
        sbcape, sbcin = cape_cin(p[i:], t[i:], td[i:], prof)
        if sbcape >= 100 * units('J/kg') and sbcin > -250 * units('J/kg'):
            pbot = p[i]
            hbot = h[i]
            bot_idx = i
            break
    if not pbot:
        return None, None

    for i in range(bot_idx + 1, p.shape[0]):
        prof = parcel_profile(p[i:], t[i], td[i])
        sbcape, sbcin = cape_cin(p[i:], t[i:], td[i:], prof)
        if sbcape < 100 * units('J/kg') or sbcin < -250 * units('J/kg'):
            ptop = p[i]
            htop = h[i]
            break

    if height_layer:
        return hbot, htop
    else:
        return pbot, ptop


###########################################
import pandas as pd
import os

rows = []
# Loop over all the files from data-0 to data-1439
for i in range(1440):
    # Construct the filename for the current file
    filename = f'data-{i}.txt'
    filepath = os.path.join('C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/sounding/Tables/', filename)

    # Read the text file into a pandas DataFrame
    col_names = ['height', 'pressure', 'temperature', 'dewpoint', 'rh', 'direction', 'speed']
    df = pd.read_csv(filepath, delim_whitespace=True, skiprows=2,  names=col_names)


    ###########################################
    # Isolate needed variables from our data file and attach units
    p = df['pressure'].values * units.hPa
    T = df['temperature'].values * units.degC
    Td = df['dewpoint'].values * units.degC
    wdir = df['direction'].values * units.degree
    sped = df['speed'].values * units.knot
    height = df['height'].values * units.meter

    ###########################################
    # Compute the wind components
    u, v = mpcalc.wind_components(sped, wdir)

    ###########################################
    # Compute common sounding index parameters
    ctotals = mpcalc.cross_totals(p, T, Td)
    kindex = mpcalc.k_index(p, T, Td)
    showalter = mpcalc.showalter_index(p, T, Td)
    total_totals = mpcalc.total_totals_index(p, T, Td)
    vert_totals = mpcalc.vertical_totals(p, T)

    ###########################################
    # Compture the parcel profile for a surface-based parcel
    prof = mpcalc.parcel_profile(p, T[0], Td[0])

    ###########################################
    # Compute the corresponding LI, CAPE, CIN values for a surface parcel
    lift_index = mpcalc.lifted_index(p, T, prof)
    cape, cin = mpcalc.cape_cin(p, T, Td, prof)

    ###########################################
    # Determine the LCL, LFC, and EL for our surface parcel
    lclp, lclt = mpcalc.lcl(p[0], T[0], Td[0])
    lfcp, _ = mpcalc.lfc(p, T, Td)
    el_pressure, _ = mpcalc.el(p, T, Td, prof)

    ###########################################
    # Compute the characteristics of a mean layer parcel (50-hPa depth)
    ml_t, ml_td = mpcalc.mixed_layer(p, T, Td, depth=50 * units.hPa)
    ml_p, _, _ = mpcalc.mixed_parcel(p, T, Td, depth=50 * units.hPa)
    mlcape, mlcin = mpcalc.mixed_layer_cape_cin(p, T, prof, depth=50 * units.hPa)

    ###########################################
    # Compute the characteristics of the most unstable parcel (50-hPa depth)
    mu_p, mu_t, mu_td, _ = mpcalc.most_unstable_parcel(p, T, Td, depth=50 * units.hPa)
    mucape, mucin = mpcalc.most_unstable_cape_cin(p, T, Td, depth=50 * units.hPa)

    ###########################################
    # Compute the Bunkers Storm Motion vector and use to calculate the critical angle
    (u_storm, v_storm), *_ = mpcalc.bunkers_storm_motion(p, u, v, height)
    critical_angle = mpcalc.critical_angle(p, u, v, height, u_storm, v_storm)

    ###########################################
    # Work on the calculations needed to compute the significant tornado parameter

    # Estimate height of LCL in meters from hydrostatic thickness
    new_p = np.append(p[p > lclp], lclp)
    new_t = np.append(T[p > lclp], lclt)
    lcl_height = mpcalc.thickness_hydrostatic(new_p, new_t)

    # Compute Surface-based CAPE
    sbcape, _ = mpcalc.surface_based_cape_cin(p, T, Td)

    # Compute SRH, given a motion vector toward the NE at 9.9 m/s
    *_, total_helicity = mpcalc.storm_relative_helicity(height, u, v, depth=1 * units.km,
                                                        storm_u=u_storm, storm_v=v_storm)

    # Copmute Bulk Shear components and then magnitude
    ubshr, vbshr = mpcalc.bulk_shear(p, u, v, height=height, depth=6 * units.km)
    bshear = mpcalc.wind_speed(ubshr, vbshr)

    # Use all computed pieces to calculate the Significant Tornado parameter
    sig_tor = mpcalc.significant_tornado(sbcape, lcl_height,
                                        total_helicity, bshear).to_base_units()

    ###########################################
    # Compute the supercell composite parameter, if possible

    # Determine the top and bottom of the effective layer using our own function
    hbot, htop = effective_layer(p, T, Td, height, height_layer=True)

    # Perform the calculation of supercell composite if an effective layer exists
    if hbot:
        esrh = mpcalc.storm_relative_helicity(height, u, v, depth=htop - hbot, bottom=hbot)
        eubshr, evbshr = mpcalc.bulk_shear(p, u, v, height=height, depth=htop - hbot, bottom=hbot)
        ebshear = mpcalc.wind_speed(eubshr, evbshr)

        super_comp = mpcalc.supercell_composite(mucape, esrh[0], ebshear)
    else:
        super_comp = np.nan

    
    # Create a dictionary of parameter names and values
    params = {
        'CAPE': [cape.magnitude, str(cape.units)],
        'CIN': [cin.magnitude, str(cin.units)],
        'LCL Pressure': [lclp.magnitude, str(lclp.units)],
        'LFC Pressure': [lfcp.magnitude, str(lfcp.units)],
        'EL Pressure': [el_pressure.magnitude, str(el_pressure.units)],
        'Lifted Index': [lift_index.magnitude, str(lift_index.units)],
        'K-Index': [kindex.magnitude, str(kindex.units)],
        'Showalter Index': [showalter.magnitude, str(showalter.units)],
        'Cross Totals': [ctotals.magnitude, str(ctotals.units)],
        'Total Totals': [total_totals.magnitude, str(total_totals.units)],
        'Vertical Totals': [vert_totals.magnitude, str(vert_totals.units)],
        'Mixed Layer - Lowest 50-hPa Temp': [ml_t.magnitude, str(ml_t.units)],
        'Mixed Layer - Lowest 50-hPa Dewp': [ml_td.magnitude, str(ml_td.units)],
        'Mixed Layer - Lowest 50-hPa CAPE': [mlcape.magnitude, str(mlcape.units)],
        'Mixed Layer - Lowest 50-hPa CIN': [mlcin.magnitude, str(mlcin.units)],
        'Most Unstable - Lowest 50-hPa Temp': [mu_t.magnitude, str(mu_t.units)],
        'Most Unstable - Lowest 50-hPa Dewp': [mu_td.magnitude, str(mu_td.units)],
        'Most Unstable - Lowest 50-hPa Pressure': [mu_p.magnitude, str(mu_p.units)],
        'Most Unstable - Lowest 50-hPa CAPE': [mucape.magnitude, str(mucape.units)],
        'Most Unstable - Lowest 50-hPa CIN': [mucin.magnitude, str(mucin.units)],
        'Bunkers Storm Motion Vector - u_storm': [u_storm.magnitude, str(u_storm.units)],
        'Bunkers Storm Motion Vector - v_storm': [v_storm.magnitude, str(v_storm.units)],
        'Critical Angle': [critical_angle.magnitude, str(critical_angle.units)],
        'Storm Relative Helicity': [total_helicity.magnitude, str(total_helicity.units)],
        'Significant Tornado Parameter': [sig_tor.magnitude, str(sig_tor.units)],
        'Supercell Composite Parameter': [super_comp.magnitude, str(super_comp.units)],
    }
    
    # Create a new row with different values
    new_row = '\t'.join([str(value[0]) for value in params.values()])
    # Append the new row to the rows list
    rows.append(new_row)

# Write the header row and all the data rows to a file
with open('C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/sounding/June_11_2022.txt', 'w') as f:
    # Write the header row
    f.write('\t'.join(params.keys()) + '\n')

    # Write the data rows
    for row in rows:
        f.write(row + '\n')



In [5]:
import pandas as pd
import os

# Load Output.txt as a dataframe
output_df = pd.read_csv('C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/sounding/June_11_2022.txt', sep='\t')


# Loop through data-0 to data-1439 files
for i in range(1,1440):
    # Construct the filename
    filename = f'C:/Users/omitu/Documents/GitHub/Urbanization-and-Climate-Change/Second_part/data/sounding/Tables/data-{i}.txt'
    
    # Check if the file exists
    if os.path.isfile(filename):
        # Load the data file as a dataframe
        data_df = pd.read_csv(filename, sep='\t')
        
        # Append the row from Output.txt to the beginning of the data dataframe
        data_df = pd.concat([output_df.iloc[i:i+1], data_df], axis=1)
        
        # Save the modified data file
        data_df.to_csv(filename, sep='\t', index=False)
